In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoderEstimator
from pyspark.sql.types import DecimalType, IntegerType, DateType
from pyspark.ml import Pipeline

In [2]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
df = spark.read.csv('./AirBnb_NYC_2019.csv',header='true',inferSchema='true')

In [4]:
df.head(1)

[Row(id='2539', name='Clean & quiet apt home by the park', host_id='2787', host_name='John', neighbourhood_group='Brooklyn', neighbourhood='Kensington', latitude='40.64749', longitude='-73.97237', room_type='Private room', price='149', minimum_nights='1', number_of_reviews='9', last_review='2018-10-19', reviews_per_month='0.21', calculated_host_listings_count='6', availability_365=365)]

In [5]:
df.select('availability_365').collect()

[Row(availability_365=365),
 Row(availability_365=355),
 Row(availability_365=365),
 Row(availability_365=194),
 Row(availability_365=0),
 Row(availability_365=129),
 Row(availability_365=0),
 Row(availability_365=220),
 Row(availability_365=0),
 Row(availability_365=188),
 Row(availability_365=6),
 Row(availability_365=39),
 Row(availability_365=314),
 Row(availability_365=333),
 Row(availability_365=0),
 Row(availability_365=46),
 Row(availability_365=321),
 Row(availability_365=12),
 Row(availability_365=21),
 Row(availability_365=249),
 Row(availability_365=0),
 Row(availability_365=347),
 Row(availability_365=364),
 Row(availability_365=304),
 Row(availability_365=233),
 Row(availability_365=85),
 Row(availability_365=0),
 Row(availability_365=75),
 Row(availability_365=311),
 Row(availability_365=67),
 Row(availability_365=355),
 Row(availability_365=255),
 Row(availability_365=284),
 Row(availability_365=359),
 Row(availability_365=269),
 Row(availability_365=340),
 Row(availabi

In [6]:
df = df.withColumn("latitude",df["latitude"].cast(DecimalType()))
df = df.withColumn("longitude",df["longitude"].cast(DecimalType()))
df = df.withColumn("price",df["price"].cast(IntegerType()))
df = df.withColumn("minimum_nights",df["minimum_nights"].cast(IntegerType()))
df = df.withColumn("number_of_reviews",df["number_of_reviews"].cast(IntegerType()))
df = df.withColumn("calculated_host_listings_count",df["calculated_host_listings_count"].cast(IntegerType()))
df = df.withColumn("last_review",df["last_review"].cast(DateType()))
df = df.withColumn("reviews_per_month",df["reviews_per_month"].cast(DecimalType()))
df = df.withColumn("availability_365",df["availability_365"].cast(IntegerType()))

In [7]:
df.columns

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [8]:
#creates indexes for each string
indexers = [StringIndexer(inputCol=col, 
                          outputCol=col+'_index').fit(df) for col in ['neighbourhood_group',
                                                                      'neighbourhood',
                                                                      'room_type']]
pipe = Pipeline(stages=indexers)
df = pipe.fit(df).transform(df)

#creates columns with 0 or 1 for each index
encoder = OneHotEncoderEstimator(inputCols=['neighbourhood_group_index','neighbourhood_index',
                                             'room_type_index'],
                                 outputCols=['neighbourhood_group_dummy','neighbourhood_dummy',
                                             'room_type_dummy'],
                                dropLast=True)
df = encoder.fit(df).transform(df)

df.dtypes

[('id', 'string'),
 ('name', 'string'),
 ('host_id', 'string'),
 ('host_name', 'string'),
 ('neighbourhood_group', 'string'),
 ('neighbourhood', 'string'),
 ('latitude', 'decimal(10,0)'),
 ('longitude', 'decimal(10,0)'),
 ('room_type', 'string'),
 ('price', 'int'),
 ('minimum_nights', 'int'),
 ('number_of_reviews', 'int'),
 ('last_review', 'date'),
 ('reviews_per_month', 'decimal(10,0)'),
 ('calculated_host_listings_count', 'int'),
 ('availability_365', 'int'),
 ('neighbourhood_group_index', 'double'),
 ('neighbourhood_index', 'double'),
 ('room_type_index', 'double'),
 ('neighbourhood_group_dummy', 'vector'),
 ('neighbourhood_dummy', 'vector'),
 ('room_type_dummy', 'vector')]

In [9]:
df.columns

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'neighbourhood_group_index',
 'neighbourhood_index',
 'room_type_index',
 'neighbourhood_group_dummy',
 'neighbourhood_dummy',
 'room_type_dummy']

In [12]:
target = 'price'
features = df.drop('id',
                     'name',
                     'host_id',
                     'host_name',
                     'neighbourhood_group_index',
                     'neighbourhood_index',
                     'room_type_index',
                     'neighbourhood_group',
                     'neighbourhood',
                     'room_type',
                  'price',
                  'last_review').columns

vector = VectorAssembler(inputCols= features,
                         outputCol='features')

vectorized_df = vector.transform(df)
vectorized_df.head(2)


[Row(id='2539', name='Clean & quiet apt home by the park', host_id='2787', host_name='John', neighbourhood_group='Brooklyn', neighbourhood='Kensington', latitude=Decimal('41'), longitude=Decimal('-74'), room_type='Private room', price=149, minimum_nights=1, number_of_reviews=9, last_review=datetime.date(2018, 10, 19), reviews_per_month=Decimal('0'), calculated_host_listings_count=6, availability_365=365, neighbourhood_group_index=1.0, neighbourhood_index=52.0, room_type_index=1.0, neighbourhood_group_dummy=SparseVector(76, {1: 1.0}), neighbourhood_dummy=SparseVector(381, {52: 1.0}), room_type_dummy=SparseVector(85, {1: 1.0}), features=SparseVector(549, {0: 41.0, 1: -74.0, 2: 1.0, 3: 9.0, 5: 6.0, 6: 365.0, 8: 1.0, 135: 1.0, 465: 1.0})),
 Row(id='2595', name='Skylit Midtown Castle', host_id='2845', host_name='Jennifer', neighbourhood_group='Manhattan', neighbourhood='Midtown', latitude=Decimal('41'), longitude=Decimal('-74'), room_type='Entire home/apt', price=225, minimum_nights=1, numb

In [13]:
train_data, test_data = vectorized_df.randomSplit([.75,.25])

lin_reg = LinearRegression(featuresCol='features', labelCol='price')

lin_reg.fit(train_data)

Py4JJavaError: An error occurred while calling o301.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 19, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<latitude_double_VectorAssembler_97a6581f9ce0:double,longitude_double_VectorAssembler_97a6581f9ce0:double,minimum_nights_double_VectorAssembler_97a6581f9ce0:double,number_of_reviews_double_VectorAssembler_97a6581f9ce0:double,reviews_per_month_double_VectorAssembler_97a6581f9ce0:double,calculated_host_listings_count_double_VectorAssembler_97a6581f9ce0:double,availability_365_double_VectorAssembler_97a6581f9ce0:double,neighbourhood_group_dummy:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,neighbourhood_dummy:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,room_type_dummy:struct<type:tinyint,size:int,indices